###Assignment 7: Game AI Using Search Algorithms
Objective: Implement AI to solve a simple turn-based game.

Problem Statement: Design an AI agent to play a game (e.g., Tic-Tac-Toe or Snake
and Ladder) using search algorithms.

Tasks:
* Use BFS and DFS for exploring game states.
* Implement A* Search with a heuristic function to improve efficiency.
* Compare search strategies for different game board configurations.

In [2]:
import time
from collections import deque
import heapq

# -----------------------
# Tic-Tac-Toe Game State
# -----------------------
class TicTacToeState:
    def __init__(self, board=None, player=1):
        self.board = board or [0]*9
        self.player = player

    def legal_moves(self):
        return [i for i, v in enumerate(self.board) if v == 0]

    def next_state(self, move):
        new_state = TicTacToeState(self.board.copy(), -self.player)
        new_state.board[move] = self.player
        return new_state

    def is_terminal(self):
        b = self.board
        lines = [
            (0,1,2),(3,4,5),(6,7,8),
            (0,3,6),(1,4,7),(2,5,8),
            (0,4,8),(2,4,6)
        ]
        for i,j,k in lines:
            s = b[i] + b[j] + b[k]
            if abs(s) == 3:
                return True
        return all(v != 0 for v in b)

    def __hash__(self):
        return hash((tuple(self.board), self.player))

    def __eq__(self, other):
        return isinstance(other, TicTacToeState) and self.player == other.player and self.board == other.board

    def __lt__(self, other):
        return heuristic(self) < heuristic(other) or (heuristic(self) == heuristic(other) and self.board < other.board)

# -----------------------
# Search Algorithms
# -----------------------
def bfs(root):
    start_time = time.time()
    queue = deque([root])
    visited = set([root])
    nodes = 0
    while queue:
        state = queue.popleft()
        nodes += 1
        if state.is_terminal():
            continue
        for move in state.legal_moves():
            child = state.next_state(move)
            if child not in visited:
                visited.add(child)
                queue.append(child)
    return {'nodes': nodes, 'time': time.time() - start_time}


def dfs(root):
    start_time = time.time()
    stack = [root]
    visited = set()
    nodes = 0
    while stack:
        state = stack.pop()
        if state in visited:
            continue
        visited.add(state)
        nodes += 1
        if not state.is_terminal():
            for move in state.legal_moves():
                stack.append(state.next_state(move))
    return {'nodes': nodes, 'time': time.time() - start_time}


def heuristic(state):
    lines = [
        (0,1,2),(3,4,5),(6,7,8),
        (0,3,6),(1,4,7),(2,5,8),
        (0,4,8),(2,4,6)
    ]
    score = 0
    for i,j,k in lines:
        tri = [state.board[x] for x in (i,j,k)]
        if tri.count(state.player) == 2 and tri.count(0) == 1:
            score -= 1
        if tri.count(-state.player) == 2 and tri.count(0) == 1:
            score += 1
    return score


def a_star(root):
    start_time = time.time()
    open_set = [(heuristic(root), 0, root)]
    g_scores = {root: 0}
    visited = set()
    nodes = 0
    while open_set:
        f, g, state = heapq.heappop(open_set)
        nodes += 1
        if state.is_terminal():
            return {'nodes': nodes, 'time': time.time() - start_time}
        if state in visited:
            continue
        visited.add(state)
        for move in state.legal_moves():
            child = state.next_state(move)
            tentative_g = g + 1
            if tentative_g < g_scores.get(child, float('inf')):
                g_scores[child] = tentative_g
                heapq.heappush(open_set, (tentative_g + heuristic(child), tentative_g, child))
    return {'nodes': nodes, 'time': time.time() - start_time}

root = TicTacToeState()
print("Tic-Tac-Toe Search Comparisons")
for name, func in [("BFS", bfs), ("DFS", dfs), ("A*", a_star)]:
    res = func(root)
    print(f"{name}: nodes={res['nodes']}, time={res['time']:.4f}s")

Tic-Tac-Toe Search Comparisons
BFS: nodes=5478, time=0.0497s
DFS: nodes=5478, time=0.0499s
A*: nodes=883, time=0.2716s


In [3]:
import time
from collections import deque
import heapq

# -----------------------
# Snakes & Ladders Game State
# -----------------------
class SnakesLadders:
    def __init__(self, transitions=None, size=100):
        self.transitions = transitions or {}
        self.size = size

    def legal_moves(self, position):
        moves = []
        for roll in range(1, 7):
            nxt = position + roll
            if nxt > self.size:
                continue
            nxt = self.transitions.get(nxt, nxt)
            moves.append(nxt)
        return moves

    def is_terminal(self, position):
        return position == self.size

# -----------------------
# Search Algorithms
# -----------------------
def bfs(game):
    start_time = time.time()
    start_pos = 1
    queue = deque([start_pos])
    visited = set([start_pos])
    nodes = 0
    while queue:
        pos = queue.popleft()
        nodes += 1
        if game.is_terminal(pos):
            break
        for nxt in game.legal_moves(pos):
            if nxt not in visited:
                visited.add(nxt)
                queue.append(nxt)
    return {'nodes': nodes, 'time': time.time() - start_time}


def dfs(game):
    start_time = time.time()
    stack = [1]
    visited = set()
    nodes = 0
    while stack:
        pos = stack.pop()
        if pos in visited:
            continue
        visited.add(pos)
        nodes += 1
        if game.is_terminal(pos):
            break
        for nxt in game.legal_moves(pos):
            stack.append(nxt)
    return {'nodes': nodes, 'time': time.time() - start_time}


def heuristic(position, size):
    return (size - position) / 6.0


def a_star(game):
    start_time = time.time()
    start = 1
    open_set = [(heuristic(start, game.size), 0, start)]
    g_scores = {start: 0}
    visited = set()
    nodes = 0
    while open_set:
        f, g, pos = heapq.heappop(open_set)
        nodes += 1
        if game.is_terminal(pos):
            return {'nodes': nodes, 'time': time.time() - start_time}
        if pos in visited:
            continue
        visited.add(pos)
        for nxt in game.legal_moves(pos):
            tentative_g = g + 1
            if tentative_g < g_scores.get(nxt, float('inf')):
                g_scores[nxt] = tentative_g
                heapq.heappush(open_set, (tentative_g + heuristic(nxt, game.size), tentative_g, nxt))
    return {'nodes': nodes, 'time': time.time() - start_time}

transitions = {4:14, 9:31, 17:7, 28:84, 40:59, 54:34, 62:19,
               63:81, 64:60, 71:91, 87:24, 93:73, 95:75, 99:78}
game = SnakesLadders(transitions)
print("Snakes & Ladders Search Comparisons")
for name, func in [("BFS", bfs), ("DFS", dfs), ("A*", a_star)]:
    res = func(game)
    print(f"{name}: nodes={res['nodes']}, time={res['time']:.4f}s")

Snakes & Ladders Search Comparisons
BFS: nodes=85, time=0.0001s
DFS: nodes=18, time=0.0000s
A*: nodes=11, time=0.0001s
